## Installations

In [ ]:
"""
conda create -n CNN python=3.10
conda activate CNN
conda install -c conda-forge cudatoolkit=11.2 cudnn=8.1.0
python -m pip install "tensorflow==2.10"
conda install Pillow
conda install scipy
conda install matplotlib
conda install -c conda-forge opencv
conda install pandas 
conda install scikit-learn
conda install "numpy<2"
"""

## Imports

In [ ]:
import os, shutil, random, glob, pandas as pd, cv2
from pathlib import Path
from ultralytics import YOLO
import torch

## Creating the DB

In [ ]:
base_dir = Path("Aircrafts")
dataset_dir = base_dir / "dataset"
splits = {"train": 0.7, "valid": 0.2, "test": 0.1}

if any((base_dir / split / "images").exists() and len(os.listdir(base_dir / split / "images")) > 0 for split in splits):
    print("One of the folders already contains files. Aborting script.")
else:
    for split in splits:
        (base_dir / split / "images").mkdir(parents=True, exist_ok=True)
        (base_dir / split / "labels").mkdir(parents=True, exist_ok=True)

    images = list(dataset_dir.glob("*.jpg"))
    random.shuffle(images)
    n = len(images)
    train_end = int(splits["train"] * n)
    valid_end = train_end + int(splits["valid"] * n)
    split_map = {
        "train": images[:train_end],
        "valid": images[train_end:valid_end],
        "test": images[valid_end:]
    }

    class_mapping = {}
    next_id = 0
    annotations = {}

    for img_path in images:
        csv_path = img_path.with_suffix(".csv")
        if not csv_path.exists():
            continue
        df = pd.read_csv(csv_path)

        im = cv2.imread(str(img_path))
        if im is None:
            continue
        h, w = im.shape[:2]
        lines = []
        for _, row in df.iterrows():
            label = str(row["class"])
            if label not in class_mapping:
                class_mapping[label] = next_id
                next_id += 1
            cls = class_mapping[label]
            xmin, ymin, xmax, ymax = row["xmin"], row["ymin"], row["xmax"], row["ymax"]
            x_c = ((xmin + xmax) / 2) / w
            y_c = ((ymin + ymax) / 2) / h
            bw = (xmax - xmin) / w
            bh = (ymax - ymin) / h
            lines.append(f"{cls} {x_c:.6f} {y_c:.6f} {bw:.6f} {bh:.6f}")
        annotations[img_path.name] = lines

    for split, imgs in split_map.items():
        for img_path in imgs:
            dst_img = base_dir / split / "images" / img_path.name
            shutil.copy(img_path, dst_img)
            label_lines = annotations.get(img_path.name, [])
            dst_label = base_dir / split / "labels" / (img_path.stem + ".txt")
            with open(dst_label, "w") as f:
                f.write("\n".join(label_lines))

    with open(base_dir / "data.yaml", "w") as f:
        f.write(f"train: {str((base_dir/'train'/'images').resolve())}\n")
        f.write(f"val: {str((base_dir/'valid'/'images').resolve())}\n")
        f.write("test: " + str((base_dir/'test'/'images').resolve()) + "\n")
        f.write(f"nc: {len(class_mapping)}\n")
        f.write("names: " + str([k for k,v in sorted(class_mapping.items(), key=lambda x: x[1])]) + "\n")

## Yolo

In [ ]:
import torch
import cv2
import os
import shutil
from ultralytics import YOLO
from pathlib import Path

In [ ]:
data_yaml = "Aircrafts/data.yaml"
output_dir = "Aircrafts/output"
test_images = "Aircrafts/test/images"
test_labels = "Aircrafts/test/labels"

Creating output folder

In [ ]:
os.makedirs(output_dir, exist_ok=True)

Using GPU

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"  # Automatically select GPU or CPU


Loading & Training The Model

In [ ]:
model = YOLO("Aircrafts/yolov5su.pt")  # Load improved pre-trained model

if not os.path.exists("Aircrafts/aircrafts.pt"):
    model.train(data=data_yaml, epochs=50, imgsz=416, name="Aircrafts", device=device, amp=False)

Tuning The Model

In [ ]:
model.tune(data=data_yaml, epochs=10, imgsz=640)

Loading Last Weights

In [ ]:
best_model_path = "runs/detect/Aircrafts/weights/best.pt"
model = YOLO(best_model_path)

In [ ]:

# Load best model after training


# Run inference on test images
results = model(test_images, save=True, save_txt=True, project=output_dir, name="test_results", device=device, conf=0.4)  # Use GPU

# Add aircraft type next to bounding boxes
def draw_boxes_with_labels(image_path, label_path, output_path):
    try:
        image = cv2.imread(image_path)
        height, width, _ = image.shape
        
        with open(label_path, "r") as f:
            labels = f.readlines()
        
        for label in labels:
            parts = label.strip().split()
            class_id = int(parts[0])
            x_center, y_center, w, h = map(float, parts[1:])
            x1 = int((x_center - w / 2) * width)
            y1 = int((y_center - h / 2) * height)
            x2 = int((x_center + w / 2) * width)
            y2 = int((y_center + h / 2) * height)
            
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(image, f"Aircraft {class_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        cv2.imwrite(output_path, image)
    except Exception as e:
        print(f"Error processing {image_path}: {e}")

# Process test images
test_images_list = list(Path(test_images).glob("*.jpg"))
for image_path in test_images_list:
    label_path = Path(test_labels) / (image_path.stem + ".txt")
    output_path = Path(output_dir) / image_path.name
    if label_path.exists():
        draw_boxes_with_labels(str(image_path), str(label_path), str(output_path))
    else:
        shutil.copy(str(image_path), str(output_path))  # Copy image if no labels exist

print("Processing complete. Check 'Aircrafts/output' for results.")